In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

import statsmodels.api as sm

In [21]:
# tomo el dataset de salida de facundo del tp1
datos = pd.read_csv('datos_salida_notebook_facundo.csv')

Index(['Unnamed: 0', 'property_type', 'lat', 'lon', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'floor', 'rooms', 'expenses', 'description', 'title', 'provincia',
       'ciudad', 'barrio', 'cochera', 'parrilla', 'patio', 'pileta', 'vista'],
      dtype='object')

In [22]:
# elimino estos 4 campos que nos habíamos olvidado de eliminar
datos.drop(['description', 'title', 'floor', 'expenses', 'Unnamed: 0'], axis=1, inplace=True)

In [23]:
datos.head(2)

,property_type,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,provincia,ciudad,barrio,cochera,parrilla,patio,pileta,vista
0,PH,-34.661824,-58.508839,62000.0,55.0,40.0,1127.272727,NaN,Capital Federal,Capital Federal,Mataderos,False,False,False,False,False
1,apartment,-34.652262,-58.522982,72000.0,55.0,55.0,1309.090909,NaN,Capital Federal,Capital Federal,Mataderos,False,False,False,False,False


In [96]:
# hago un primer análisis
datos.groupby(['provincia', 'ciudad', 'property_type'])['property_type'].count().sort_values(ascending=False)


provincia                     ciudad            property_type
Capital Federal               Capital Federal   apartment        22620
Buenos Aires Costa Atlántica  Mar del Plata     apartment         4383
Bs.As. G.B.A. Zona Norte      Tigre             house             4229
                                                apartment         3577
Santa Fe                      Rosario           apartment         3301
                                                                 ...  
Córdoba                       Villa Icho Cruz   house                1
                                                store                1
                              Villa Los Aromos  house                1
                                                store                1
Chubut                        Rada Tilly        apartment            1
Name: property_type, Length: 639, dtype: int64

In [97]:
# elijo solo deptos porque casa y phs no están dentro de los más importantes y las 4 zonas del amba
Masc_prov_prop_type = (((datos.provincia == 'Bs.As. G.B.A. Zona Norte') | (datos.provincia == 'Bs.As. G.B.A. Zona Oeste') |\
                        (datos.provincia == 'Bs.As. G.B.A. Zona Sur') | (datos.provincia == 'Capital Federal')) &\
                        (datos.property_type == 'apartment'))
datos_gba = datos[Masc_prov_prop_type]
datos_gba.shape[0] / datos.shape[0]
# de los 85969 registros originales, usamos 38806 -> casi el 45%

0.4514005211241392

In [105]:
# agrupo y ordeno x provincia ciudad y barrio
datos_gba.groupby(['provincia', 'ciudad', 'barrio'])['property_type'].count().sort_values(ascending=False)

provincia                 ciudad             barrio                       
Capital Federal           Capital Federal    Palermo                          3030
                                             Belgrano                         2259
                                             Caballito                        1847
Bs.As. G.B.A. Zona Norte  Tigre              Nordelta                         1733
Capital Federal           Capital Federal    Recoleta                         1289
                                                                              ... 
Bs.As. G.B.A. Zona Norte  Pilar              El Molino                           1
Bs.As. G.B.A. Zona Oeste  General Rodríguez  General Rodríguez                   1
Bs.As. G.B.A. Zona Norte  Pilar              Country Farm Club                   1
Bs.As. G.B.A. Zona Oeste  Hurlingham         Hurlingham                          1
Bs.As. G.B.A. Zona Norte  Pilar              Barrio Cerrado Las Casuarinas       1
Name: proper

In [132]:
# Genero un dataframe que da true or false según la cuenta de propiedades sea > 200
df_masc_mayor_cantidad = pd.DataFrame(datos_gba.groupby(['provincia', 'ciudad', 'barrio'])['property_type'].count()> 200)
df_masc_mayor_cantidad_final = df_masc_mayor_cantidad.reset_index(['provincia', 'ciudad', 'barrio'])
df_masc_mayor_cantidad_final.rename(columns={'property_type': 'mayor'}, inplace=True)
df_masc_mayor_cantidad_final


,provincia,ciudad,barrio,mayor
0,Bs.As. G.B.A. Zona Norte,Escobar,Belén de Escobar,False
1,Bs.As. G.B.A. Zona Norte,Escobar,Garín,False
2,Bs.As. G.B.A. Zona Norte,Escobar,Maschwitz,False
3,Bs.As. G.B.A. Zona Norte,Escobar,Septiembre,False
4,Bs.As. G.B.A. Zona Norte,General San Martín,Barrio Parque General San Martin,False
...,...,...,...,...
249,Capital Federal,Capital Federal,Villa Riachuelo,False
250,Capital Federal,Capital Federal,Villa Santa Rita,False
251,Capital Federal,Capital Federal,Villa Soldati,False
252,Capital Federal,Capital Federal,Villa Urquiza,True


In [133]:
# uno los dos dataframe para que agregue una columna mayor que sea true cuando pertenece al grupo donde hay más de 200 propiedades
datos_gba_etiq = pd.merge(datos_gba, df_masc_mayor_cantidad_final, on=['provincia', 'ciudad', 'barrio'], how='left')

In [134]:
datos_gba_etiq.head(3)

,property_type,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,rooms,provincia,ciudad,barrio,cochera,parrilla,patio,pileta,vista,mayor
0,apartment,-34.652262,-58.522982,72000.0,55.0,55.0,1309.090909,NaN,Capital Federal,Capital Federal,Mataderos,False,False,False,False,False,False
1,apartment,-34.559873,-58.443362,138000.0,45.0,40.0,3066.666667,NaN,Capital Federal,Capital Federal,Belgrano,False,False,False,False,False,True
2,apartment,-34.559873,-58.443362,195000.0,65.0,60.0,3000.000000,NaN,Capital Federal,Capital Federal,Belgrano,False,False,False,False,False,True


In [137]:
# si mayor es False etiqueta ciudad como baja_cant
datos_gba_etiq['ciudad'] = datos_gba_etiq.apply(lambda x: x['ciudad'] if x['mayor'] else 'baja_cant', axis=1)

In [138]:
# si mayor es False etiquieta barrio como baja_cant
datos_gba_etiq['barrio'] = datos_gba_etiq.apply(lambda x: x['barrio'] if x['mayor'] else 'baja_cant', axis=1)

In [146]:
# concateno provincia y ciudad
datos_gba_etiq['prov_ciud_barr'] = datos_gba_etiq['provincia'].str.cat(datos_gba_etiq['ciudad'], sep=' ')

In [149]:
# agrego barrio
datos_gba_etiq['prov_ciud_barr'] = datos_gba_etiq['prov_ciud_barr'].str.cat(datos_gba_etiq['barrio'], sep=' ')

In [154]:
pd.get_dummies(datos_gba_etiq['prov_ciud_barr'], drop_first=True, prefix='p_c_b')

,p_c_b_Bs.As. G.B.A. Zona Norte San Isidro Martínez,p_c_b_Bs.As. G.B.A. Zona Norte Tigre Nordelta,p_c_b_Bs.As. G.B.A. Zona Norte Tigre Tigre,p_c_b_Bs.As. G.B.A. Zona Norte Vicente López Florida,p_c_b_Bs.As. G.B.A. Zona Norte Vicente López La Lucila,p_c_b_Bs.As. G.B.A. Zona Norte Vicente López Olivos,p_c_b_Bs.As. G.B.A. Zona Norte Vicente López Vicente López,p_c_b_Bs.As. G.B.A. Zona Norte baja_cant baja_cant,p_c_b_Bs.As. G.B.A. Zona Oeste La Matanza Ramos Mejía,p_c_b_Bs.As. G.B.A. Zona Oeste Morón Morón,...,p_c_b_Capital Federal Capital Federal Recoleta,p_c_b_Capital Federal Capital Federal Saavedra,p_c_b_Capital Federal Capital Federal San Cristobal,p_c_b_Capital Federal Capital Federal San Telmo,p_c_b_Capital Federal Capital Federal Villa Crespo,p_c_b_Capital Federal Capital Federal Villa Devoto,p_c_b_Capital Federal Capital Federal Villa Luro,p_c_b_Capital Federal Capital Federal Villa Urquiza,p_c_b_Capital Federal Capital Federal Villa del Parque,p_c_b_Capital Federal baja_cant baja_cant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38801,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
38802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38803,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
